# Discovering knowledge in customer shopping behaviors
### Course: DAMI330484_22_2_01
### Instructor: M.Sc. Nguyen Van Thanh
| Group 19         |          |
|:-----------------|:---------|
| Đỗ Hoàng Thịnh   | 20133122 |
| Nguyễn Minh Tiến | 20133093 |
| Huỳnh Nguyễn Tín | 20133094 |
| Bùi Lê Hải Triều | 20133101 |

### 1. Dataset
Nhóm sử dụng tập dữ liệu chứa thông tin giao dịch của khách hàng từ 10 trung tâm mua sắm lớn tại đất nước Istanbul, từ năm 2021 đến thời điểm hiện tại năm 2023 trên [Kaggle](https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset). Ngoài thông tin giao dịch, tập dữ liệu cũng cung cấp thông tin về độ tuổi, giới tính, phù hợp với nghiệp vụ khai phá.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
transactions = pd.read_csv('data/transactions.csv')
transactions.info()

Tập dữ liệu có 99457 giao dịch và 10 cột.

| Attribute      | Description                       | Example                       | Data type   |
|:---------------|:----------------------------------|:------------------------------|:------------|
| invoice_no     | Mã giao dịch                      | I138884                       | Categorical |
| customer_id    | Mã khách hàng                     | C241288                       | Categorical |
| gender         | Giới tính                         | Male, Female                  | Categorical |
| age            | Độ tuổi                           | 18, 69                        | Numerical   |
| category       | Danh mục sản phẩm                 | Clothing                      | Categorical |
| quantity       | Số lượng sản phẩm trong giao dịch | 1, 5                          | Numerical   |
| price          | Đơn giá sản phẩm trong giao dịch  | 1500.4                        | Numerical   |
| payment_method | Phương thức thanh toán            | Cash, Credit Card, Debit Card | Categorical |
| invoice_date   | Ngày diễn ra giao dịch            | 5/8/2022                      | Categorical |
| shopping_mall  | Địa điểm diễn ra giao dịch        | Kanyon                        | Categorical |

In [ ]:
transactions.sample(5)

In [ ]:
transactions.isnull().sum()

In [ ]:
transactions.duplicated().sum()

Tập dữ liệu không chứa giá trị null ở bất kỳ cột nào và không có giao dịch trùng lặp.

### 2. Data preparation
Để phục vụ việc khai phá về sau, nhóm sẽ tạo cột mới chứa thông tin tổng số tiền thanh toán trên mỗi giao dịch.

In [ ]:
transactions['total'] = transactions['quantity'] * transactions['price']
transactions.sample(5)

Nhóm cũng sẽ thực hiện nhóm tuổi khách hàng thành 6 độ tuổi để giảm độ nhiễu của tập dữ liệu: 18 đến 24, 25 đến 34, 35 đến 44, 45 đến 54, 55 đến 64, và 65 đến 70.

In [ ]:
bins = [18, 24, 34, 44, 54, 64, 70]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-70']
transactions['age_group'] = pd.cut(transactions['age'], bins=bins, labels=labels)
age_group_type = pd.CategoricalDtype(labels, ordered=True)
transactions['age_group'] = transactions['age_group'].astype(age_group_type)
transactions.drop('age', axis=1, inplace=True)
transactions.sample(5)

Nhóm có thể giảm lượng dữ liệu qua việc loại bỏ cột không mang ý nghĩa khai phá như mã giao dịch và mã khách hàng.

In [ ]:
transactions.duplicated(subset=['invoice_no']).any()

In [ ]:
transactions.duplicated(subset=['customer_id']).any()

Tập dữ liệu không có giao dịch với cùng mã giao dịch hoặc cùng mã khách hàng. Điều này có nghĩa mỗi khách hàng chỉ thực hiện giao dịch một lần. Vì vậy, nhóm có thể loại bỏ hai cột này.

In [ ]:
transactions.drop(['invoice_no', 'customer_id'], axis=1, inplace=True)
transactions.sample(5)

Kiểm tra số lượng giao dịch trùng lặp sau khi loại bỏ hai cột trên.

In [ ]:
transactions.duplicated().sum()

In [ ]:
transactions.drop_duplicates(keep='first')

### 3. EDA
Trước khi thực hiện việc khai phá dữ liệu, nhóm sẽ thực hiện phân tích sơ bộ tập dữ liệu hiện tại thông qua biểu đồ trực quan để hiểu hơn về nghiệp vụ trước khi thực hiện khai phá.

In [ ]:
import seaborn as sns
import plotly.express as px

#### 3.1. Category wise
Đầu tiên, danh mục sản phẩm phổ biến nhất trên tổng số lượng sản phẩm trong mỗi giao dịch.

In [ ]:
category = transactions.groupby('category')['quantity'].sum()
category = pd.DataFrame({'category': category.index, 'quantity': category.values})
category['categories'] = 'categories'

fig = px.treemap(category, path=['categories', 'category'], values='quantity', color='quantity',
                 hover_data=['category'], color_continuous_scale='Blues')
fig.update_layout(width=1000, height=600, paper_bgcolor='LightSteelBlue')
fig.show(renderer='notebook')

Như vậy, sản phẩm thuộc danh mục Clothing, Cosmetics, và Food and Beverage xuất hiện nhiều nhất trong toàn bộ số giao dịch.

#### 3.2. Gender wise
Đáng lưu ý, Clothing và Cosmetics là hai danh mục sản phẩm trên thực tế thường được mua bởi phụ nữ, nên có thể số lượng khách hàng nữ cao hơn nam.

In [ ]:
transactions['gender'].value_counts()

Với số lượng khách hàng nữ cao hơn gần 20000, doanh thu có thể phần lớn đến từ khách hàng nữ.

In [ ]:
gender = transactions.groupby('gender')['total'].sum()
gender = pd.DataFrame({'gender': gender.index, 'total': gender.values})

fig = px.pie(gender, values='total', names='gender')
fig.update_layout(paper_bgcolor='LightSteelBlue')
fig.show(renderer='notebook')

Đúng như dự đoán, gần 60% doanh thu đến từ khách hàng nữ.

In [ ]:
gender_category = transactions.groupby(['gender', 'category'])['total'].sum().unstack().reset_index()

fig = px.bar(gender_category,
             x=['Books', 'Clothing', 'Cosmetics', 'Food and Beverage', 'Shoes', 'Souvenir', 'Technology', 'Toys'],
             y='gender')
fig.update_layout(width=1000, height=600, plot_bgcolor='LightSteelBlue', paper_bgcolor='LightSteelBlue',
                  legend=dict(title='category'))
fig.show(renderer='notebook')

Với mỗi danh mục sản phẩm, khách hàng nữ đều chi nhiều hơn khách hàng nam khi mua sắm. Tuy nhiên, đây cũng có thể là vì số lượng khách hàng nữ cao hơn.
Vì vậy, nhóm không thể dựa vào biểu đồ trực quan như trên để đưa ra quyết định nghiệp vụ marketing hoặc xây dựng hệ thống recommendation. Thay vào đó, để đưa ra chiến lược nhằm duy trì mối quan hệ khách hàng chính xác và hiệu quả, nhóm cần thực hiện quá trình khai phá dữ liệu.

### 4. Data mining
Mục tiêu chính của nhóm là xác định phân khúc khách hàng thân thiết hoặc sản phẩm có giá trị doanh nghiệp cao dựa trên thuật toán phân cụm (Clustering) và phân loại (Classification). Ngoài ra, thuật toán kết hợp (Associate) cũng sẽ được sử dụng để phân tích hành vi mua hàng của khách hàng và xu hướng, khuôn mẫu có ích cho quyết định nghiệp vụ.

#### 4.2. Classification
Phân loại là quá trình gồm hai bước: learning và predicting. Trong bước learning, mô hình phân loại được hình thành sử dụng tập dữ liệu training. Trong bước predicting, mô hình trên sẽ được sử dụng để đưa ra dự đoán dựa trên đầu vào. Phân loại, khác với phân cụm, là mô hình học máy có giám sát.
Nhóm sẽ sử dụng thuật toán Decision Tree vì thuật toán dễ trực quan hóa và dễ hiểu.

In [ ]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus

pd.options.mode.chained_assignment = None

##### 4.2.1. Selecting features
Do Decision Tree là mô hình học máy có giám sát, nhóm sẽ xác định biến giải thích (feature variables) và biến kết quả (target variables) trong nghiệp vụ phân loại giới tính khách hàng.

In [ ]:
features = ['age_group', 'category', 'quantity', 'payment_method', 'total']
targets = ['gender']
X = transactions[features]
y = transactions[targets]

##### 4.2.2. Transforming data
Với hệ thống học máy có nền tảng mạnh, cột có kiểu phân loại được xử lý một cách tự nhiên như ngôn ngữ R sẽ sử dụng factors, hoặc Weka sẽ sử dụng kiểu nominal.
Mô hình Decision Tree nhóm sử dụng từ thư viện scikit-learn chỉ chấp nhận biến giải thích (feature variables) kiểu số và liên tục (continuous numerical variables).
Để chuyển đổi kiểu dữ liệu, nhóm có hai lựa chọn: one-hot-encoding và label-encoding. Tuy nhiên, khi sử dụng label-encoding trên một cột, mô hình học máy có thể vô tình xem cột đó có thứ tự hoặc cấp bậc. Nhóm có thể mong muốn việc này với cột độ tuổi, tuy nhiên, cột danh mục sản phẩm và phương thức thanh toán không nên có.
Sử dụng label-encoding trên cột độ tuổi.

In [ ]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(X.age_group)
label_encoder.classes_

Thay thế cột độ tuổi ban đầu.

In [ ]:
X['age_group'] = label_encoder.fit_transform(X['age_group'])
X.sample(5)

Sử dụng one-hot-encoding trên cột danh mục sản phẩm và phương thức thanh toán.

In [ ]:
X = pd.get_dummies(X, columns=['category', 'payment_method'])
X.sample(5)

Ngoài ra, biến kết quả (target variables) giới tính cũng nên được áp dụng label-encoding.

In [ ]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(y.gender)
label_encoder.classes_

Thay thế cột giới tính ban đầu.

In [ ]:
y['gender'] = pd.get_dummies(y, columns=['gender'])
y.sample(5)

##### 4.2.3. Splitting data
Để xét độ chính xác của mô hình, nhóm sẽ chia tập dữ liệu thành tập dữ liệu dành cho training và tập dữ liệu dành cho testing. Nhóm sẽ dành ra 70% giao dịch từ tập dữ liệu ban đầu cho việc training và 30% cho việc testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

##### 4.2.4. Building model
Bắt đầu với việc fit tập dữ liệu training vào mô hình Decision Tree.

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

Tiếp theo, dự đoán biến kết quả (target variables) với đầu vào là tập dữ liệu testing X chứa biến giải thích (feature variables).

In [ ]:
y_pred = clf.predict(X_test)
y_pred[:5]

##### 4.2.5. Evaluating model
Xét độ chính xác của mô hình bằng phương pháp so sánh giữa tập dữ liệu testing y chứa biến kết quả (target variables) và tập dữ liệu dự đoán trên.

In [ ]:
metrics.accuracy_score(y_test, y_pred)

##### 4.2.6. Improving accuracy
Hyper-parameters là tham số có thể định nghĩa lúc xây dựng mô hình học máy. Với Decision Tree, việc cấu hình quy luật thuật toán phân chia dữ liệu (theo entropy hay gini impurity) hoặc chiều sâu tối đa có thể giúp tăng độ chính xác của mô hình và tránh overfitting.
Việc tìm ra tổ hợp tham số tốt nhất cho mô hình có thể được tự động hóa sử dụng GridSearchCV. Đầu tiên, nhóm sẽ xác định tham số nhóm muốn thay đổi. GridSearchCV thực hiện cross-validation đối với từng tổ hợp tham số trên và xác định tổ hợp tham số tốt nhất.

In [ ]:
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8] + list(range(1, 10)),
    'splitter': ['best', 'random']
}

clf = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=params, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)
clf.best_params_

Sử dụng tổ hợp tham số tốt nhất GridSearchCV tìm được để xây dựng lại mô hình.

In [ ]:
clf = DecisionTreeClassifier(criterion=clf.best_params_['criterion'], splitter=clf.best_params_['splitter'],
                             max_depth=clf.best_params_['max_depth'], max_features=clf.best_params_['max_features'])
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

Với tổ hợp tham số mới, độ chính xác của mô hình tăng nhẹ và mô hình không còn bị overfitting.

##### 4.2.7. Visualizing model
Biểu đồ trực quan Decision Tree cho thấy cấu trúc mô hình học máy với mỗi ô chữ nhật là một nút. Nội dung một nút cho biết quy luật thuật toán phân chia dữ liệu tại dòng đầu tiên và biến kết quả (target variables) tại dòng cuối cùng.

In [ ]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True, special_characters=True, feature_names=X.columns,
                class_names=['0', '1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('gender.png')
dot_data = StringIO()
Image(graph.create_png())